# I32 : Use \_\_getattr\_\_, \_\_getattribute\_\_, and \_\_setattr\_\_ for Lazy Attributes

In [5]:
class LazyDB(object):
    def __init__(self):
        self.exists = 5
        
    def __getattr__(self, name):
        value = 'Value for %s' % name
        setattr(self, name, value)
        return value

In [6]:
data = LazyDB()
print(data.__dict__)
print(data.foo)
print(data.__dict__)

{'exists': 5}
Value for foo
{'exists': 5, 'foo': 'Value for foo'}


In [7]:
class LoggingLazyDB(LazyDB):
    def __getattr__(self, name):
        print('Called __getattr__(%s)' % name)
        return super().__getattr__(name)

In [8]:
data = LoggingLazyDB()
print('exists:', data.exists)
print('foo:   ', data.foo)
print('foo:   ', data.foo)

exists: 5
Called __getattr__(foo)
foo:    Value for foo
foo:    Value for foo


In [11]:
class ValidatingDB(object):
    def __init__(self):
        self.exists = 5
        
    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        try:
            return super().__getattribute__(name)
        except AttributeError:
            value = 'Value for %s' % name
            setattr(self, name, value)
            return value

In [12]:
data = ValidatingDB()
print('exists:', data.exists)
print('foo:   ', data.foo)
print('foo:   ', data.foo)

Called __getattribute__(exists)
exists: 5
Called __getattribute__(foo)
foo:    Value for foo
Called __getattribute__(foo)
foo:    Value for foo


In [13]:
class MissingPropertyDB(object):
    def __getattr__(self, name):
        if name == 'bad_name':
            raise AttributeError('%s is missing' % name)

In [14]:
data = MissingPropertyDB()
data.bad_name

AttributeError: bad_name is missing

- Python	code	implementing	generic	functionality	often	relies	on	the	hasattr	built-in function	to	determine	when	properties	exist,	and	the	getattr	built-in	function	to retrieve	property	values.	

In [15]:
data = LoggingLazyDB()
print(data.__dict__)
print(hasattr(data, 'foo'))
print(data.__dict__)

{'exists': 5}
Called __getattr__(foo)
True
{'exists': 5, 'foo': 'Value for foo'}


- in case of \_\_getattribute\_\_ method called each time hasattr or getattr is run on an object

In [16]:
data = ValidatingDB()
print(hasattr(data, 'foo'))
print(hasattr(data, 'foo'))

Called __getattribute__(foo)
True
Called __getattribute__(foo)
True


- The	\_\_setattr\_\_	method	is	always	called	every	time	an	attribute	is	assigned	on	an instance	(either	directly	or	through	the	setattr	built-in	function). 

In [17]:
class SavingDB(object):
    def __setattr__(self, name, value):
        # Save some data to the DB log
        # ...
        super().__setattr__(name, value)
        
        
class LoggingSavingDB(SavingDB):
    def __setattr__(self, name, value):
        print('Called __setattr__(%s, %r)' % (name, value))
        super().__setattr__(name, value)

In [18]:
data = LoggingSavingDB()
print(data.__dict__)
data.foo = 5
print(data.__dict__)
data.foo = 7
print(data.__dict__)

{}
Called __setattr__(foo, 5)
{'foo': 5}
Called __setattr__(foo, 7)
{'foo': 7}


In [19]:
class BrokenDictionaryDB(object):
    def __init__(self, data):
        self_data = {}
        
    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        return self._data[name]

In [20]:
# recursice reference
data = BrokenDictionaryDB({'foo': 3})
data.foo

Called __getattribute__(foo)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __getattribute__(_data)
Called __g

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/dockeruser/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-cc294e2cef0f>", line 3, in <module>
    data.foo
  File "<ipython-input-19-fe9dbb5efef0>", line 7, in __getattribute__
    return self._data[name]
  File "<ipython-input-19-fe9dbb5efef0>", line 7, in __getattribute__
    return self._data[name]
  File "<ipython-input-19-fe9dbb5efef0>", line 7, in __getattribute__
    return self._data[name]
  [Previous line repeated 651 more times]
  File "<ipython-input-19-fe9dbb5efef0>", line 6, in __getattribute__
    print('Called __getattribute__(%s)' % name)
  File "/home/dockeruser/anaconda3/lib/python3.6/site-packages/ipykernel/iostream.py", line 350, in write
    is_child = (not self._is_master_process())
  File "/home/dockeruser/anaconda3/lib/python3.6/site-packages/ipykernel/iostream.py", line 285, in _is_mast

RecursionError: maximum recursion depth exceeded in comparison

- ** The problem is tha \_\_getattribute\_\_ accesses self.\_data, which causes \_\_getattribute\_\_ to run again, which accesses self.\_data again, and so on. The solution is to use the super().\_\_getattribute\_\_ method on your instance to fetch values from the instance attribute dictionary. This avoids the recursion. **

In [21]:
class DictionaryDB(object):
    def __init__(self, data):
        self._data = data
        
    def __getattribute__(self, name):
        data_dict = super().__getattribute__('_data')
        return data_dict[name]

- Similarly,	you’ll	need	\_\_setattr\_\_	methods	that	modify	attributes	on	an	object	to	use super().\_\_setattr\_\_. 

# I33: Validate Subclasses with Metaclasses 

- One of the simplest applicatios of metaclasses is verifying that a class was defined correctly. When you're building a complex class hierarchy, you may want to enforce style, require overriding methods, or have strict relationships between class attributes. Metaclasses enable these use cases by providing a reliable way to run your validation code each time a new subclass is defined.

- Often	a	class’s	validation	code	runs	in	the	\_\_init\_\_	method,	when	an	object	of	the class’s	type	is	constructed. Using	metaclasses	for	validation	can	raise	errors	much earlier.

- Before	I	get	into	how	to	define	a	metaclass	for	validating	subclasses,	it’s	important	to understand	the	metaclass	action	for	standard	objects.	A	metaclass	is	defined	by	inheriting from	type.	In	the	default	case,	a	metaclass	receives	the	contents	of	associated	class statements	in	its	\_\_new\_\_	method.	Here,	you	can	modify	the	class	information	before	the type	is	actually	constructed: 

In [22]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print((meta, name, bases, class_dict))
        return type.__new__(meta, name, bases, class_dict)
    

class MyClass(object, metaclass=Meta):
    stuff = 123
    
    def foo(self):
        pass

(<class '__main__.Meta'>, 'MyClass', (<class 'object'>,), {'__module__': '__main__', '__qualname__': 'MyClass', 'stuff': 123, 'foo': <function MyClass.foo at 0x7fa9d7d9dea0>})


In [23]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # Don't validate the abstract Polygon class
        if bases != (object,):
            if class_dict['sides'] > 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)
    

class Polygon(object, metaclass=ValidatePolygon):
    sides = None # 서브클래스에 의해 정의됨

    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180
    
    
class Triangle(Polygon):
    sides = 3

- If you try to define a polygon with fewer than three sides, the validation will cause the class statement to fail immediately after the class statement body. This means your program will not even be able to start running when you define such a class.

In [ ]:
print('Before')
class Line(Polygon):
    print('Before sides')
    sides = 1
    print('After sides')

# I34 : Register Class Existence with Metaclasses

- Another common use of metaclasses is to automatically register types in your program.

- Registration is useful for doing reverse lookups, where you need to map a simple identifier back to a corresponding class.

In [27]:
import json

class Serializable(object):
    def __init__(self, *args):
        self.args = args
        
    def serialize(self):
        return json.dumps({'args': self.args})

In [28]:
# immutable 2D
class Point2D(Serializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y
        
    def __repr__(self):
        return 'Point@D(%d, %d)' % (self.x, self.y)

In [29]:
point = Point2D(5, 3)
print('Object:    ', point)
print('Serialized:', point.serialize())

Object:     Point@D(5, 3)
Serialized: {"args": [5, 3]}


In [30]:
class Deserializable(Serializable):
    @classmethod
    def deserialize(cls, json_data):
        params = json.loads(json_data)
        return cls(*params['args'])

In [32]:
class BetterPoint2D(Deserializable):
    #...
    pass

In [35]:
point = BetterPoint2D(5, 3)
print('B', point)
data = point.serialize()
print('S', data)
after = BetterPoint2D.deserialize(data)
print('A', after)

B <__main__.BetterPoint2D object at 0x7fa9d7e41048>
S {"args": [5, 3]}
A <__main__.BetterPoint2D object at 0x7fa9d7e41080>


In [36]:
class BetterSerializable(object):
    def __init__(self, *args):
        self.args = args
        
    def serialize(self):
        return json.dumps({
            'class': self.__class__.__name__,
            'args': self.args,
        })
    
    def __repr__(self):
        pass

    

# P.112 